In [ ]:
# default_exp train.general

# General training methods

> Various general methods to be used during training

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export 
import pandas as pd
import numpy as np
from fastai.learner import Recorder
from fastprogress.fastprogress import format_time
from csv import writer
import time
#from fastai.callback.tensorboard import TensorBoardCallback, tensorboard_log
from fastcore.imports import *
from fastai.callback.tracker import ReduceLROnPlateau, TrackerCallback
from fastcore.basics import *

In [ ]:
# export
def pp_dls(dl, show_tfms:bool=False):
    '''Pretty print dl contents'''

    # each dataloader has datasets (not dataset, note the plural)
    # I think it is usally two, one for x (image) and one for y (label) with much overlap between the two
    dsx = dl.dataset.tls[0]
    dsy = dl.dataset.tls[1]

    # get df for each (obviously requires that a df was used to define the datablock)
    dss_df = dsx.items # (could also be dxy, does not matter)

    # get x and y labels 
    x_label = dsx.tfms.col_reader.cols.items[0]
    y_label = dsy.tfms.col_reader.cols.items[0]

    # pp
    print(f"No. Images:\t {dl.n}")
    print(f"Classes:\t {dss_df[y_label].unique()}")
    print(f"Shuffle:\t {dl.shuffle}")
    print(f"Bs:\t\t {dl.bs}\n")
    if show_tfms:
        print("Transformations\n")
        for i, tfm in enumerate(dl.after_batch.fs, 1):
            print(f"\t{i}.) {tfm.name}")

In [ ]:
# export 
class SilentRecorder(Recorder):

    def __init__(self, csv_file, add_time=True, train_metrics=False, valid_metrics=True, beta=0.98):
        super().__init__(add_time, train_metrics, valid_metrics, beta)
        self.csv_file = f"{csv_file}.csv"
        open(self.csv_file, "w").close()
        
    def before_fit(self):
        super().before_fit()
        self.append_to_csv(self.metric_names)
        
    def after_epoch(self):
        "Store and log the loss/metric values without verbose printing"
        self.learn.final_record = self.log[1:].copy()
        self.values.append(self.learn.final_record)
        if self.add_time: self.log.append(format_time(time.time() - self.start_epoch))
        self.iters.append(self.smooth_loss.count)
        self.append_to_csv(self.log)

    def after_fit(self):
        self.append_to_csv([])
        
    def append_to_csv(self, row):
        with open(self.csv_file, 'a') as f:
            writer_object = writer(f)
            writer_object.writerow(row)
            f.close()    

In [ ]:
# export
# class MultiFitTensorBoardCallback(TensorBoardCallback):
#     "Saves model topology, losses & metrics for tensorboard and tensorboard projector during training"
#     def __init__(self, log_dir=None, trace_model=True, log_preds=True, n_preds=9, projector=False, layer=None):
#         super().__init__(log_dir, trace_model, log_preds, n_preds, projector, layer)
#         self.num_fit = -1
#         self.custom_iter = 0
        
#     def before_fit(self):
#         super().before_fit()
#         self.learn.train_iter = self.custom_iter
#         self.num_fit += 1

#     def after_batch(self):
#         self.writer.add_scalar('train_loss', self.smooth_loss, (self.num_fit*self.custom_iter)+self.train_iter)
#         for i,h in enumerate(self.opt.hypers):
#             for k,v in h.items(): self.writer.add_scalar(f'{k}_{i}', v, (self.num_fit*self.custom_iter)+self.train_iter)

#     def after_epoch(self):
#         for n,v in zip(self.recorder.metric_names[2:-1], self.recorder.log[2:-1]):
#             self.writer.add_scalar(n, v, (self.num_fit*self.custom_iter)+self.train_iter)
#         if self.log_preds:
#             b = self.dls.valid.one_batch()
#             self.learn.one_batch(0, b)
#             preds = getattr(self.loss_func, 'activation', noop)(self.pred)
#             out = getattr(self.loss_func, 'decodes', noop)(preds)
#             x,y,its,outs = self.dls.valid.show_results(b, out, show=False, max_n=self.n_preds)
#             tensorboard_log(x, y, its, outs, self.writer, (self.num_fit*self.custom_iter)+self.train_iter)
            
#     def after_fit(self):
#         self.custom_iter = self.learn.train_iter

In [ ]:
# export
class ReduceLROnPlateauNTimes(ReduceLROnPlateau):
    def __init__(self, n, monitor='valid_loss', comp=None, min_delta=0., patience=1, factor=10., min_lr=0, reset_on_fit=True):
        super().__init__( monitor=monitor, comp=comp, min_delta=min_delta, patience=patience, factor=factor, min_lr=min_lr, reset_on_fit=reset_on_fit)
        self.n = n
        self.counter = 0
        self.alive = True
        
    def after_epoch(self):
        "Compare the value monitored to its best score and reduce LR by `factor` if no improvement."
        TrackerCallback.after_epoch(self)
        if self.new_best: self.wait = 0
        elif self.counter < self.n:
            self.wait += 1
            if self.wait >= self.patience:
                self.counter += 1
                old_lr = self.opt.hypers[-1]['lr']
                for h in self.opt.hypers: h['lr'] = max(h['lr'] / self.factor, self.min_lr)
                self.wait = 0
                if self.opt.hypers[-1]["lr"] < old_lr:
                    print(f'Epoch {self.epoch}: reducing lr to {self.opt.hypers[-1]["lr"]}')
        else:
            if self.alive:
                print(f"Epoch {self.epoch}: Killing callback '{self.__class__.__name__}' as already reduced {self.counter} time(s)")
                self.alive = False